<a href="https://colab.research.google.com/github/CakeNuthep/Create_Knowledge_Graph/blob/main/NRMS_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
path_file =  '/content/drive/MyDrive/KnowledgeGraph/Json_researcher/nrms.json'

In [4]:
f = open(path_file, "r",encoding="utf8")
count = 0
content = []
string_json = ''
begin = '['
end = ']'
split = ','

string_json += begin
while(True):
    # Read a line.
    line = f.readline()
    # When readline returns an empty string, the file is fully read.
    if line == "":
        print("::DONE::")
        break
    if count != 0:
      string_json += split
    string_json += line
    content.append(line)
    count +=1

string_json += end

::DONE::


In [5]:
import json
data = json.loads(string_json)

In [6]:
df = pd.json_normalize(data)

In [7]:
df

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01


# separate nrms

In [8]:
df.columns

Index(['_index', '_type', '_id', '_score', '_source.link', '_source.name',
       '_source.id', '_source.author', '_source.start', '_source.stop',
       '_source.kind', '_source.group', '_source.word', '_source.goal',
       '_source.branch', '_source.status', '_source.budget',
       '_source.department', '_source.pullDate'],
      dtype='object')

In [9]:
def is_english(str):
    for i in str:
        if i in ('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'):
            return True
    else:    
        return False

def is_thai(str):
    for i in str:
        if i in ('ก','ข','ฃ','ค','ฅ','ฆ','ง','จ','ฉ','ช','ซ','ฌ','ญ','ฎ','ฏ','ฐ','ฑ','ฒ','ณ','ด','ต','ถ','ท','ธ','น','บ','ป','ผ','ฝ','พ','ฟ','ภ','ม','ย','ร','ล','ว','ศ','ษ','ส','ห','ฬ','อ','ฮ'):
            return True
    else:
        return False

def language(text):
    if is_english(text):
      return 'eng'
    elif is_thai(text):
      return 'th'
    else:
      return 'other'

def strip_row(text):
    text = text.strip()
    text = ' '.join(text.split())
    return text

def first_name(list_text):
    name = ''
    if len(list_text) > 0:
      name = strip_row(list_text[0])
    return name

def check_first_author(row):
    if pd.isna(row['first_name']):
      return 'no'
    elif pd.isna(row['new_name']):
      return 'no'
    elif row['first_name'] == '':
      return 'no'
    elif row['first_name'] == row['new_name']:
      return 'yes'
    else:
      return 'no'

In [10]:
df_new_author = df.assign(new_name=df['_source.author'].str.split(","))

In [11]:
df_new_author

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,[นายศิวกร อ่างทอง]
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,[ผศ. อำนวย เรืองวารี]
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,[นายประสิทธิ์ ศรีนคร]
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,[ผศ.ดร. ชัยยะ ปราณีตพลกรัง]
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,[รศ. ดร. จตุรงค์ ลังกาพินธุ์]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,[นายอดิศักดิ์ จำปาทอง]
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,[นายบุญธง วสุริย์]
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,[ผศ. อลิสา เลี้ยงรื่นรมย์]
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,[นายนายสุรพล จักรชัยกุล]


In [12]:
df_new_author['first_name'] = df_new_author['new_name'].apply(first_name)
df_new_author = df_new_author.explode('new_name')

In [13]:
df_new_author

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล


In [14]:
df_new = pd.concat([df_new_author])

In [15]:
df_new

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล


In [16]:
df_new['language'] = df_new['new_name'].apply(language)
df_new['new_name'] = df_new['new_name'].apply(strip_row)

In [17]:
df_new

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,th
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,th
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,th
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,th
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,th
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,th
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,th
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,th


In [18]:
df_new['first_author'] = df_new.apply(check_first_author,axis=1) 

In [19]:
df_new

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,th,yes
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,th,yes
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,th,yes
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,th,yes
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,th,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,th,yes
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,th,yes
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,th,yes
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,th,yes


In [20]:
df_thai = df_new[df_new['language']=='th']
df_eng = df_new[df_new['language']=='eng']
df_other = df_new[df_new['language']=='other']

In [21]:
df_thai

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,th,yes
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,th,yes
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,th,yes
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,th,yes
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,th,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27309,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,th,yes
27310,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,th,yes
27311,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,th,yes
27312,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,th,yes


In [22]:
df_eng

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author
317,nrms,nrms,29416,1,,การใช้โปรไบโอติกที่พบในปลาเพื่อเป็นทางเลือกแทน...,29416,Assist. Prof. Dr.,1/10/2563,30/9/2564,โครงการย่อย ภายใต้ชุดโครงการวิจัย,,"โพรไบโอติก, ปลานิล, Streptococcus agalactiae",The main objective of this study is to investi...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเชียงใหม่,2019-10-24,Assist. Prof. Dr.,Assist. Prof. Dr.,eng,yes
318,nrms,nrms,29419,1,,การพัฒนาและการผลิตวัคซีนทางปากเพื่อการเลี้ยงปล...,29419,Dr. Hien Van Doan,1/10/2561,30/9/2562,โครงการย่อย ภายใต้ชุดโครงการวิจัย,,"วัคซีนทางปาก, ปลานิล, Streptococcus agalactiae",The main objective of the study is to investig...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเชียงใหม่,2019-10-24,Dr. Hien Van Doan,Dr. Hien Van Doan,eng,yes
962,nrms,nrms,2006,1,256109A3060066,การย้ายอาณาเขตของลูกนกและความสามารถในการผลิตลู...,2006,นายAndrew Pierce,1/10/2559,30/9/2561,โครงการ,เปลี่ยนแปลงเล็กน้อย,การรอดของลูกนก การย้ายอาณาเขตของลูกนก การติดวิ...,i). estimate juvenile survivorship of understo...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-24,นายAndrew Pierce,นายAndrew Pierce,eng,yes
2360,nrms,nrms,254239,1,2560A30402173,การศึกษาการทำงานของเบตากลูโคไซเดสจากข้าวที่มีผ...,254239,Prof.Dr. James Ketudat-Cairns,1/10/2557,30/9/2560,โครงการ,เปลี่ยนแปลงเล็กน้อย,"ข้าว,ฮอร์โมนพืช,เบตากลูโคไซเดส,กลูโคส เอสเตอร์...",The purpose of this project is to identify and...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีสุรนารี,2019-10-24,Prof.Dr. James Ketudat-Cairns,Prof.Dr. James Ketudat-Cairns,eng,yes
2498,nrms,nrms,255170,1,2560A10102005,การพัฒนาวัสดุเคลือบอัลลอยคอมพอสิตโลหะผสมนิกเกิ...,255170,Dr. เจียเชียน ฉิน,1/10/2558,30/9/2560,โครงการ,เปลี่ยนแปลงเล็กน้อย,"เคลือบนิกเกิล-ทังสเตน-โบรอน,การชุบไฟฟ้าแบบร่วม...",The present proposal aims at finding a replace...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",รอการทำสัญญา,0,จุฬาลงกรณ์มหาวิทยาลัย,2019-10-24,Dr. เจียเชียน ฉิน,Dr. เจียเชียน ฉิน,eng,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24831,nrms,nrms,256630,1,2560A30602204,การย้ายอาณาเขตของลูกนกและความสามารถในการผลิตลู...,256630,นายAndrew Pierce,1/10/2559,30/9/2561,โครงการ,ไม่เปลี่ยนแปลง,"Copsychus malabaricus,การย้ายอาณาเขตของลูกนก,อ...",1. estimate juvenile survivorship of understor...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น (ต่อเนื่องที่ดำเนินการต่...,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-24,นายAndrew Pierce,นายAndrew Pierce,eng,yes
25195,nrms,nrms,270003,1,2560A10702130,การพัฒนาสารประกอบเรืองแสงทรีส-ไซโคลเมทาเลทเออร...,270003,Dr. ฟิลลิป เคียล่าร์,1/10/2559,30/9/2560,โครงการ,เปลี่ยนแปลงเล็กน้อย,"กล้องจุลทรรศน์แสงฟลูออเรสเซ็นต์,การย้อมสีเซลล์...",- To develop triscyclometalated iridium comple...,",สาขาวิทยาศาสตร์เคมีและเภสัช",โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยนเรศวร,2019-10-31,Dr. ฟิลลิป เคียล่าร์,Dr. ฟิลลิป เคียล่าร์,eng,yes
25950,nrms,nrms,186540,1,2559A10602081,ความหลากหลายทางสมบัติของสารประกอบโคออร์ดิเนชัน...,186540,Assist. Prof กิตติพงศ์ ไชยนอก,1/10/2558,30/9/2559,โครงการ,เปลี่ยนแปลงเล็กน้อย,"การวิศวกรรมผลึก,สารประกอบโคออร์ดิเนชันพอลิเมอร...",1) เพื่อออกแบบและสังเคราะห์ประกอบโคออร์ดิเนชัน...,",สาขาวิทยาศาสตร์เคมีและเภสัช",โครงการวิจัยเสร็จสิ้น,0,มหาวิทยาลัยธรรมศาสตร์,2019-10-31,Assist. Prof กิตติพงศ์ ไชยนอก,Assist. Prof กิตติพงศ์ ไชยนอก,eng,yes
26071,nrms,nrms,197270,1,2559A11103045,การประยุกต์ใช้เทคโนโลยีไบโอฟล็อคในบ่อเลี้ยงปลา...,197270,Dr. สุดาพร ตงศิริ,1/10/2557,30/9/2559,โครงการย่อย ภายใต้ชุดโครงการวิจัย,เปลี่ยนแปลงเล็กน้อย,ไบโอฟลอค;การเลี้ยงปลาเศรษฐกิจแบบรวม;ปลาพื้นเมื...,6.2. ปีที่ 2 เพื่อสำรวจความต้องการการพัฒนาการเ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,มหาวิทยาลัยแม่โจ้,2019-10-31,Dr. สุดาพร ตงศิริ,Dr. สุดาพร ตงศิริ,eng,yes


In [23]:
df_other

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author
46,nrms,nrms,11775,1,,สร้างและพัฒนาศักยภาพของเจ้าหน้าที่กรมควบคุมโรค...,11775,,1/10/2561,30/9/2563,โครงการ,,วัณโรค วิจัย เครือข่ายวิจัย การพัฒนาศักยภาพ กา...,Training and workshop to increase TB research ...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยมหิดล,2019-10-24,,,other,no
78,nrms,nrms,12698,1,,การพัฒนาเครื่องปรับระดับเมนิสคัสแบบอัตโนมัติ,12698,,1/10/2560,30/9/2562,โครงการ,,เครื่องแก้ววัดปริมาตร ปิเปต เมนิสคัส การประมวล...,เพื่อพัฒนาชุดควบคุมการปรับระดับเมนิสคัสแบบอัตโ...,,โครงการอยู่ระหว่างดำเนินการ,0,กรมวิทยาศาสตร์บริการ,2019-10-24,,,other,no
95,nrms,nrms,16150,1,,การยกระดับโครงสร้างพื้นฐานความปลอดภัยทางชีวภาพ...,16150,,1/10/2561,30/9/2563,โครงการ,,"ความปลอดภัยทางชีวภาพ, ห้องปฏิบัติการวิจัย, โคร...",1) เพื่อประเมินโครงสร้างพื้นฐานของห้องปฏิบัติก...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยมหิดล,2019-10-24,,,other,no
111,nrms,nrms,16384,1,,โครงการพัฒนามาตรการปฏิบัติต่อผู้กระทำผิดอาญาแท...,16384,,1/11/2561,30/7/2562,โครงการ,,มาตรการปฏิบัติต่อผู้กระทำผิดอาญาแทนการควบคุมตัว,๑. เพื่อศึกษาสภาพปัญหาของมาตรการปฏิบัติต่อผู้ก...,,โครงการอยู่ระหว่างดำเนินการ,0,สำนักงานกิจการยุติธรรม,2019-10-24,,,other,no
154,nrms,nrms,17505,1,,โครงการยกระดับมาตรฐานเครื่องมือทางการแพทย์,17505,,1/10/2561,30/9/2565,โครงการ,,"ทดสอบ, สอบเทียบ, เครื่องมือแพทย์",4.1 เพื่อพัฒนาเทคนิควิธีการทดสอบและสอบเทียบที่...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,สถาบันวิจัยวิทยาศาสตร์และเทคโนโลยีแห่งประเทศไทย,2019-10-24,,,other,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24312,nrms,nrms,5934,1,,การจัดตั้งห้องปฏิบัติการตรวจวิเคราะห์อายุด้วยร...,5934,,,,โครงการ,โครงการที่เข้าใหม่,,เพื่อการพัฒนาการวิเคราะห์ปริมาณ C-14 ที่นำไปสู...,,โครงการอยู่ระหว่างดำเนินการ,0,สถาบันเทคโนโลยีนิวเคลียร์แห่งชาติ (องค์การมหาชน),2019-10-24,,,other,no
25500,nrms,nrms,271519,1,2560A00602003,การพัฒนาการเพาะเลี้ยงปลากระแห (Barbonymus schw...,271519,,1/10/2559,30/9/2560,โครงการ,ไม่เปลี่ยนแปลง,"ปลากระแห,ชีววิทยาปลา,การเพาะเลี้ยงปลา",เพื่อพัฒนาการเพาะเลี้ยงปลากระแห เพื่อการอนุรัก...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการอยู่ระหว่างดำเนินการ,0,สำนักงานคณะกรรมการการอาชีวศึกษา,2019-10-31,,,other,no
27183,nrms,nrms,626770,1,2559NRCT72022,โครงการวิจัยความหลากหลายทางชีวภาพของพืชผักและส...,626770,,1/10/2558,30/9/2559,โครงการ,โครงการที่เข้าใหม่,,สำรวจ รวบรวมความหลากหลายทางชีวภาพของสายพันธุ์พ...,",สาขาวิทยาศาสตร์กายภาพและคณิตศาสตร์",โครงการวิจัยเสร็จสิ้น,0,กองบริหารแผนและงบประมาณการวิจัย (กบง.),2019-11-01,,,other,no
27208,nrms,nrms,544939,1,,ยุทธศาสตร์การพัฒนากลุ่มวิจัยเพื่อการพัฒนาเชิงพ...,544939,,1/10/2559,31/1/2561,โครงการ,ไม่เปลี่ยนแปลง,ระบบบริหารจัดการงานวิจัยเพื่อการพัฒนาสังคมและช...,เพื่อหนุนเสริมและเพิ่มขีดความสามารถของเกษตรกร ...,,โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,,,other,no


# create Author Table

In [24]:
uniq_name = df_new['new_name'].unique()

In [25]:
uniq_name

array(['นายศิวกร อ่างทอง', 'ผศ. อำนวย เรืองวารี', 'นายประสิทธิ์ ศรีนคร',
       ..., 'นางนัทธีรัตน์ พีระพันธุ์', 'นายอดิศักดิ์ จำปาทอง',
       'นายฉลอง ทับศรี'], dtype=object)

In [26]:
df_unique_name = pd.DataFrame(uniq_name,columns=['new_name'])

In [27]:
df_unique_name

,new_name
0,นายศิวกร อ่างทอง
1,ผศ. อำนวย เรืองวารี
2,นายประสิทธิ์ ศรีนคร
3,ผศ.ดร. ชัยยะ ปราณีตพลกรัง
4,รศ. ดร. จตุรงค์ ลังกาพินธุ์
...,...
13654,นายจักรกฤษณ์ พจนศิลป์
13655,นายเติมเพชร สุขคณาภิบาล
13656,นางนัทธีรัตน์ พีระพันธุ์
13657,นายอดิศักดิ์ จำปาทอง


In [28]:
start_id = 534267
df_unique_name['id'] = df_unique_name.index + start_id

In [29]:
df_unique_name

,new_name,id
0,นายศิวกร อ่างทอง,534267
1,ผศ. อำนวย เรืองวารี,534268
2,นายประสิทธิ์ ศรีนคร,534269
3,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,534270
4,รศ. ดร. จตุรงค์ ลังกาพินธุ์,534271
...,...,...
13654,นายจักรกฤษณ์ พจนศิลป์,547921
13655,นายเติมเพชร สุขคณาภิบาล,547922
13656,นางนัทธีรัตน์ พีระพันธุ์,547923
13657,นายอดิศักดิ์ จำปาทอง,547924


In [30]:
result_thai = pd.merge(df_thai,
                       df_unique_name,
                       left_on='new_name',
                       right_on='new_name',
                       how='left')

result_eng = pd.merge(df_eng,
                       df_unique_name,
                       left_on='new_name',
                       right_on='new_name',
                       how='left')

result_other = pd.merge(df_other,
                       df_unique_name,
                       left_on='new_name',
                       right_on='new_name',
                       how='left')

result_all = pd.merge(df_new,
                       df_unique_name,
                       left_on='new_name',
                       right_on='new_name',
                       how='left')

In [31]:
result_thai

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author,id
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,th,yes,534267
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,th,yes,534268
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,th,yes,534269
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,th,yes,534270
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,th,yes,534271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26543,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,th,yes,547924
26544,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,th,yes,536246
26545,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,th,yes,541420
26546,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,th,yes,536886


In [32]:
result_eng

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author,id
0,nrms,nrms,29416,1,,การใช้โปรไบโอติกที่พบในปลาเพื่อเป็นทางเลือกแทน...,29416,Assist. Prof. Dr.,1/10/2563,30/9/2564,โครงการย่อย ภายใต้ชุดโครงการวิจัย,,"โพรไบโอติก, ปลานิล, Streptococcus agalactiae",The main objective of this study is to investi...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเชียงใหม่,2019-10-24,Assist. Prof. Dr.,Assist. Prof. Dr.,eng,yes,534565
1,nrms,nrms,29419,1,,การพัฒนาและการผลิตวัคซีนทางปากเพื่อการเลี้ยงปล...,29419,Dr. Hien Van Doan,1/10/2561,30/9/2562,โครงการย่อย ภายใต้ชุดโครงการวิจัย,,"วัคซีนทางปาก, ปลานิล, Streptococcus agalactiae",The main objective of the study is to investig...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเชียงใหม่,2019-10-24,Dr. Hien Van Doan,Dr. Hien Van Doan,eng,yes,534566
2,nrms,nrms,2006,1,256109A3060066,การย้ายอาณาเขตของลูกนกและความสามารถในการผลิตลู...,2006,นายAndrew Pierce,1/10/2559,30/9/2561,โครงการ,เปลี่ยนแปลงเล็กน้อย,การรอดของลูกนก การย้ายอาณาเขตของลูกนก การติดวิ...,i). estimate juvenile survivorship of understo...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-24,นายAndrew Pierce,นายAndrew Pierce,eng,yes,535120
3,nrms,nrms,254239,1,2560A30402173,การศึกษาการทำงานของเบตากลูโคไซเดสจากข้าวที่มีผ...,254239,Prof.Dr. James Ketudat-Cairns,1/10/2557,30/9/2560,โครงการ,เปลี่ยนแปลงเล็กน้อย,"ข้าว,ฮอร์โมนพืช,เบตากลูโคไซเดส,กลูโคส เอสเตอร์...",The purpose of this project is to identify and...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีสุรนารี,2019-10-24,Prof.Dr. James Ketudat-Cairns,Prof.Dr. James Ketudat-Cairns,eng,yes,536305
4,nrms,nrms,255170,1,2560A10102005,การพัฒนาวัสดุเคลือบอัลลอยคอมพอสิตโลหะผสมนิกเกิ...,255170,Dr. เจียเชียน ฉิน,1/10/2558,30/9/2560,โครงการ,เปลี่ยนแปลงเล็กน้อย,"เคลือบนิกเกิล-ทังสเตน-โบรอน,การชุบไฟฟ้าแบบร่วม...",The present proposal aims at finding a replace...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",รอการทำสัญญา,0,จุฬาลงกรณ์มหาวิทยาลัย,2019-10-24,Dr. เจียเชียน ฉิน,Dr. เจียเชียน ฉิน,eng,yes,536409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,nrms,nrms,256630,1,2560A30602204,การย้ายอาณาเขตของลูกนกและความสามารถในการผลิตลู...,256630,นายAndrew Pierce,1/10/2559,30/9/2561,โครงการ,ไม่เปลี่ยนแปลง,"Copsychus malabaricus,การย้ายอาณาเขตของลูกนก,อ...",1. estimate juvenile survivorship of understor...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น (ต่อเนื่องที่ดำเนินการต่...,0,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,2019-10-24,นายAndrew Pierce,นายAndrew Pierce,eng,yes,535120
57,nrms,nrms,270003,1,2560A10702130,การพัฒนาสารประกอบเรืองแสงทรีส-ไซโคลเมทาเลทเออร...,270003,Dr. ฟิลลิป เคียล่าร์,1/10/2559,30/9/2560,โครงการ,เปลี่ยนแปลงเล็กน้อย,"กล้องจุลทรรศน์แสงฟลูออเรสเซ็นต์,การย้อมสีเซลล์...",- To develop triscyclometalated iridium comple...,",สาขาวิทยาศาสตร์เคมีและเภสัช",โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยนเรศวร,2019-10-31,Dr. ฟิลลิป เคียล่าร์,Dr. ฟิลลิป เคียล่าร์,eng,yes,543750
58,nrms,nrms,186540,1,2559A10602081,ความหลากหลายทางสมบัติของสารประกอบโคออร์ดิเนชัน...,186540,Assist. Prof กิตติพงศ์ ไชยนอก,1/10/2558,30/9/2559,โครงการ,เปลี่ยนแปลงเล็กน้อย,"การวิศวกรรมผลึก,สารประกอบโคออร์ดิเนชันพอลิเมอร...",1) เพื่อออกแบบและสังเคราะห์ประกอบโคออร์ดิเนชัน...,",สาขาวิทยาศาสตร์เคมีและเภสัช",โครงการวิจัยเสร็จสิ้น,0,มหาวิทยาลัยธรรมศาสตร์,2019-10-31,Assist. Prof กิตติพงศ์ ไชยนอก,Assist. Prof กิตติพงศ์ ไชยนอก,eng,yes,540258
59,nrms,nrms,197270,1,2559A11103045,การประยุกต์ใช้เทคโนโลยีไบโอฟล็อคในบ่อเลี้ยงปลา...,197270,Dr. สุดาพร ตงศิริ,1/10/2557,30/9/2559,โครงการย่อย ภายใต้ชุดโครงการวิจัย,เปลี่ยนแปลงเล็กน้อย,ไบโอฟลอค;การเลี้ยงปลาเศรษฐกิจแบบรวม;ปลาพื้นเมื...,6.2. ปีที่ 2 เพื่อสำรวจความต้องการการพัฒนาการเ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,มหาวิทยาลัยแม่โจ้,2019-10-31,Dr. สุดาพ

In [33]:
result_other

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author,id
0,nrms,nrms,11775,1,,สร้างและพัฒนาศักยภาพของเจ้าหน้าที่กรมควบคุมโรค...,11775,,1/10/2561,30/9/2563,โครงการ,,วัณโรค วิจัย เครือข่ายวิจัย การพัฒนาศักยภาพ กา...,Training and workshop to increase TB research ...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยมหิดล,2019-10-24,,,other,no,534308
1,nrms,nrms,12698,1,,การพัฒนาเครื่องปรับระดับเมนิสคัสแบบอัตโนมัติ,12698,,1/10/2560,30/9/2562,โครงการ,,เครื่องแก้ววัดปริมาตร ปิเปต เมนิสคัส การประมวล...,เพื่อพัฒนาชุดควบคุมการปรับระดับเมนิสคัสแบบอัตโ...,,โครงการอยู่ระหว่างดำเนินการ,0,กรมวิทยาศาสตร์บริการ,2019-10-24,,,other,no,534308
2,nrms,nrms,16150,1,,การยกระดับโครงสร้างพื้นฐานความปลอดภัยทางชีวภาพ...,16150,,1/10/2561,30/9/2563,โครงการ,,"ความปลอดภัยทางชีวภาพ, ห้องปฏิบัติการวิจัย, โคร...",1) เพื่อประเมินโครงสร้างพื้นฐานของห้องปฏิบัติก...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยมหิดล,2019-10-24,,,other,no,534308
3,nrms,nrms,16384,1,,โครงการพัฒนามาตรการปฏิบัติต่อผู้กระทำผิดอาญาแท...,16384,,1/11/2561,30/7/2562,โครงการ,,มาตรการปฏิบัติต่อผู้กระทำผิดอาญาแทนการควบคุมตัว,๑. เพื่อศึกษาสภาพปัญหาของมาตรการปฏิบัติต่อผู้ก...,,โครงการอยู่ระหว่างดำเนินการ,0,สำนักงานกิจการยุติธรรม,2019-10-24,,,other,no,534308
4,nrms,nrms,17505,1,,โครงการยกระดับมาตรฐานเครื่องมือทางการแพทย์,17505,,1/10/2561,30/9/2565,โครงการ,,"ทดสอบ, สอบเทียบ, เครื่องมือแพทย์",4.1 เพื่อพัฒนาเทคนิควิธีการทดสอบและสอบเทียบที่...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,สถาบันวิจัยวิทยาศาสตร์และเทคโนโลยีแห่งประเทศไทย,2019-10-24,,,other,no,534308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,nrms,nrms,5934,1,,การจัดตั้งห้องปฏิบัติการตรวจวิเคราะห์อายุด้วยร...,5934,,,,โครงการ,โครงการที่เข้าใหม่,,เพื่อการพัฒนาการวิเคราะห์ปริมาณ C-14 ที่นำไปสู...,,โครงการอยู่ระหว่างดำเนินการ,0,สถาบันเทคโนโลยีนิวเคลียร์แห่งชาติ (องค์การมหาชน),2019-10-24,,,other,no,534308
702,nrms,nrms,271519,1,2560A00602003,การพัฒนาการเพาะเลี้ยงปลากระแห (Barbonymus schw...,271519,,1/10/2559,30/9/2560,โครงการ,ไม่เปลี่ยนแปลง,"ปลากระแห,ชีววิทยาปลา,การเพาะเลี้ยงปลา",เพื่อพัฒนาการเพาะเลี้ยงปลากระแห เพื่อการอนุรัก...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการอยู่ระหว่างดำเนินการ,0,สำนักงานคณะกรรมการการอาชีวศึกษา,2019-10-31,,,other,no,534308
703,nrms,nrms,626770,1,2559NRCT72022,โครงการวิจัยความหลากหลายทางชีวภาพของพืชผักและส...,626770,,1/10/2558,30/9/2559,โครงการ,โครงการที่เข้าใหม่,,สำรวจ รวบรวมความหลากหลายทางชีวภาพของสายพันธุ์พ...,",สาขาวิทยาศาสตร์กายภาพและคณิตศาสตร์",โครงการวิจัยเสร็จสิ้น,0,กองบริหารแผนและงบประมาณการวิจัย (กบง.),2019-11-01,,,other,no,534308
704,nrms,nrms,544939,1,,ยุทธศาสตร์การพัฒนากลุ่มวิจัยเพื่อการพัฒนาเชิงพ...,544939,,1/10/2559,31/1/2561,โครงการ,ไม่เปลี่ยนแปลง,ระบบบริหารจัดการงานวิจัยเพื่อการพัฒนาสังคมและช...,เพื่อหนุนเสริมและเพิ่มขีดความสามารถของเกษตรกร ...,,โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,,,other,no,534308


In [34]:
result_all

,_index,_type,_id,_score,_source.link,_source.name,_source.id,_source.author,_source.start,_source.stop,_source.kind,_source.group,_source.word,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,first_author,id
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,th,yes,534267
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,th,yes,534268
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,th,yes,534269
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,th,yes,534270
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,th,yes,534271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27310,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,th,yes,547924
27311,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,th,yes,536246
27312,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,th,yes,541420
27313,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,th,yes,536886


# Create Publish

## Create Publish thai

In [35]:
result_thai.columns

Index(['_index', '_type', '_id', '_score', '_source.link', '_source.name',
       '_source.id', '_source.author', '_source.start', '_source.stop',
       '_source.kind', '_source.group', '_source.word', '_source.goal',
       '_source.branch', '_source.status', '_source.budget',
       '_source.department', '_source.pullDate', 'new_name', 'first_name',
       'language', 'first_author', 'id'],
      dtype='object')

In [36]:
result_thai = result_thai.rename(columns={"id": "auth_ID", "first_author": "firstAuthor","_id":"source_ID","_source.id":"paper_ID","_source.name":"paperTitle","_source.word":"paperKeywords","_source.link":"citation"})

In [37]:
result_thai

,_index,_type,source_ID,_score,citation,paperTitle,paper_ID,_source.author,_source.start,_source.stop,_source.kind,_source.group,paperKeywords,_source.goal,_source.branch,_source.status,_source.budget,_source.department,_source.pullDate,new_name,first_name,language,firstAuthor,auth_ID
0,nrms,nrms,7072,1,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,นายศิวกร อ่างทอง,1/10/2561,30/9/2562,ชุดโครงการวิจัย,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,6.1 เพื่อออกแบบการซอยหัวหอมแดงจาก เครื่องซอยหั...,,ส่งกลับให้นักวิจัยแก้ไขโครงการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,นายศิวกร อ่างทอง,นายศิวกร อ่างทอง,th,yes,534267
1,nrms,nrms,7073,1,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ผศ. อำนวย เรืองวารี,1/10/2561,30/9/2562,โครงการ,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,1. ศึกษากระบวนการและหลักการถ่ายภาพด้วยกล้องถ่า...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ. อำนวย เรืองวารี,ผศ. อำนวย เรืองวารี,th,yes,534268
2,nrms,nrms,6712,1,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,นายประสิทธิ์ ศรีนคร,1/10/2561,30/9/2562,โครงการ,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",6.1 สร้างเครื่องย่อยทางปาล์มน้ำมันและเครื่องอั...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย,2019-10-24,นายประสิทธิ์ ศรีนคร,นายประสิทธิ์ ศรีนคร,th,yes,534269
3,nrms,nrms,7087,1,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,1/10/2561,30/9/2562,โครงการ,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",2.1 เพื่อออกแบบและสร้างเครื่องอบแห้งปลาด้วยแสง...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,th,yes,534270
4,nrms,nrms,9502,1,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,รศ. ดร. จตุรงค์ ลังกาพินธุ์,1/10/2561,30/9/2562,โครงการ,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",เพื่อออกแบบและพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวงท...,,โครงการอยู่ระหว่างดำเนินการ,0,มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี,2019-10-24,รศ. ดร. จตุรงค์ ลังกาพินธุ์,รศ. ดร. จตุรงค์ ลังกาพินธุ์,th,yes,534271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26543,nrms,nrms,640818,1,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,นายอดิศักดิ์ จำปาทอง,,,โครงการย่อย ภายใต้ชุดโครงการวิจัย,ไม่เปลี่ยนแปลง,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,1. เพื่อศึกษาพฤติกรรมการดูแลตนเองของผู้สูงอายุ...,",สาขาเทคโนโลยีสารสนเทศและนิเทศศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายอดิศักดิ์ จำปาทอง,นายอดิศักดิ์ จำปาทอง,th,yes,547924
26544,nrms,nrms,303080,1,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,นายบุญธง วสุริย์,1/10/2557,30/9/2558,ชุดโครงการวิจัย,โครงการที่เข้าใหม่,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,1) เพื่อวิจัยและพัฒนาชุดชุดถังหมักคุณภาพสูงควบ...,",สาขาเกษตรศาสตร์และชีววิทยา",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายบุญธง วสุริย์,นายบุญธง วสุริย์,th,yes,536246
26545,nrms,nrms,303082,1,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,ผศ. อลิสา เลี้ยงรื่นรมย์,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,1. เพื่อศึกษาบริบทของการผลิตข้าวไร่ในแปลงยางพา...,",สาขาเศรษฐศาสตร์",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,ผศ. อลิสา เลี้ยงรื่นรมย์,ผศ. อลิสา เลี้ยงรื่นรมย์,th,yes,541420
26546,nrms,nrms,383660,1,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,นายนายสุรพล จักรชัยกุล,1/10/2557,30/9/2558,โครงการ,โครงการที่เข้าใหม่,,1 เพื่อศึกษาวิเคราะห์อิทธิพลของส่วนผสมทางเคมีท...,",สาขาวิศวกรรมศาสตร์และอุตสาหกรรมวิจัย",โครงการวิจัยเสร็จสิ้น,0,สำนักงานกองทุนสนับสนุนการวิจัย (สกว.),2019-11-01,นายนายสุรพล จักรชัยกุล,นายนายสุรพล จักรชัยกุล,th,yes,536886


In [38]:
result_thai.columns

Index(['_index', '_type', 'source_ID', '_score', 'citation', 'paperTitle',
       'paper_ID', '_source.author', '_source.start', '_source.stop',
       '_source.kind', '_source.group', 'paperKeywords', '_source.goal',
       '_source.branch', '_source.status', '_source.budget',
       '_source.department', '_source.pullDate', 'new_name', 'first_name',
       'language', 'firstAuthor', 'auth_ID'],
      dtype='object')

In [39]:
result_thai = result_thai.drop(['_index', '_type', '_score','first_name',
       '_source.author','_source.start','_source.stop','_source.kind','_source.group',
       '_source.goal','_source.branch','_source.status','_source.budget','_source.department',
       '_source.pullDate','new_name','language'],axis=1)

In [40]:
result_thai

,source_ID,citation,paperTitle,paper_ID,paperKeywords,firstAuthor,auth_ID
0,7072,,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,7072,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,yes,534267
1,7073,,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,7073,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,yes,534268
2,6712,,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,6712,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",yes,534269
3,7087,,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,7087,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",yes,534270
4,9502,,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,9502,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",yes,534271
...,...,...,...,...,...,...,...
26543,640818,2558000013130,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,640818,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,yes,547924
26544,303080,2558000111024,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,303080,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,yes,536246
26545,303082,2558000112271,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,303082,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,yes,541420
26546,383660,2558000112589,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,383660,,yes,536886


In [41]:
result_thai['sourceName'] = 'nrms'
result_thai['paperType'] = 'project'

# result_thai['paper_ID']=''
# result_thai['paperTitle']=''
result_thai['paperAbtract_th'] = ''
result_thai['paperAbtract_en'] = ''
# result_thai['paperKeywords'] = ''
result_thai['paperPage'] = ''
result_thai['publishedDate'] = ''
result_thai['publishedYear'] = ''
result_thai['publisherName'] = ''
result_thai['bookName'] = ''
result_thai['bookNo'] = ''
result_thai['bookVolume'] = ''

result_thai['bookEditor'] = ''
result_thai['bookSeries'] = ''
result_thai['Fund'] = ''
result_thai['DOI'] = ''
result_thai['ISSN'] = ''
# result_thai['citation'] = ''
result_thai['refURL']=''
result_thai['crossRef'] = ''

In [42]:
all_column =  ["auth_ID","firstAuthor","sourceName","source_ID","paper_ID","paperTitle","paperType","paperAbtract_th","paperAbtract_en","paperKeywords","paperPage","publishedDate","publishedYear","publisherName","bookName","bookNo","bookVolume","bookEditor","bookSeries","Fund","DOI","ISSN","citation","refURL","crossRef"]
len(all_column)

25

In [43]:
len(result_thai.columns)

25

In [44]:
result_thai = result_thai[all_column]

In [45]:
result_thai

,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef
0,534267,yes,nrms,7072,7072,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,project,,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,,,,,,,,,,,,,,,
1,534268,yes,nrms,7073,7073,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,project,,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,,,,,,,,,,,,,,,
2,534269,yes,nrms,6712,6712,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,project,,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",,,,,,,,,,,,,,,
3,534270,yes,nrms,7087,7087,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,project,,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",,,,,,,,,,,,,,,
4,534271,yes,nrms,9502,9502,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,project,,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26543,547924,yes,nrms,640818,640818,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,project,,,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,,,,,,,,,,,,,2558000013130,,
26544,536246,yes,nrms,303080,303080,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,project,,,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,,,,,,,,,,,,,2558000111024,,
26545,541420,yes,nrms,303082,303082,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,project,,,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,,,,,,,,,,,,,2558000112271,,
26546,536886,yes,nrms,383660,383660,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,project,,,,,,,,,,,,,,,,2558000112589,,


## Create Publish ENG

In [46]:
result_eng = result_eng.rename(columns={"id": "auth_ID", "first_author": "firstAuthor","_id":"source_ID","_source.id":"paper_ID","_source.name":"paperTitle","_source.word":"paperKeywords","_source.link":"citation"})

In [47]:
result_eng = result_eng.drop(['_index', '_type', '_score','first_name',
       '_source.author','_source.start','_source.stop','_source.kind','_source.group',
       '_source.goal','_source.branch','_source.status','_source.budget','_source.department',
       '_source.pullDate','new_name','language'],axis=1)

In [48]:
result_eng['sourceName'] = 'nrms'
result_eng['paperType'] = 'project'

# result_eng['paper_ID']=''
# result_eng['paperTitle']=''
result_eng['paperAbtract_th'] = ''
result_eng['paperAbtract_en'] = ''
# result_eng['paperKeywords'] = ''
result_eng['paperPage'] = ''
result_eng['publishedDate'] = ''
result_eng['publishedYear'] = ''
result_eng['publisherName'] = ''
result_eng['bookName'] = ''
result_eng['bookNo'] = ''
result_eng['bookVolume'] = ''

result_eng['bookEditor'] = ''
result_eng['bookSeries'] = ''
result_eng['Fund'] = ''
result_eng['DOI'] = ''
result_eng['ISSN'] = ''
# result_eng['citation'] = ''
result_eng['refURL']=''
result_eng['crossRef'] = ''

In [49]:
all_column =  ["auth_ID","firstAuthor","sourceName","source_ID","paper_ID","paperTitle","paperType","paperAbtract_th","paperAbtract_en","paperKeywords","paperPage","publishedDate","publishedYear","publisherName","bookName","bookNo","bookVolume","bookEditor","bookSeries","Fund","DOI","ISSN","citation","refURL","crossRef"]
len(all_column)

25

In [50]:
len(result_thai.columns)

25

In [51]:
result_eng = result_eng[all_column]

In [52]:
result_eng.head()

,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef
0,534565,yes,nrms,29416,29416,การใช้โปรไบโอติกที่พบในปลาเพื่อเป็นทางเลือกแทน...,project,,,"โพรไบโอติก, ปลานิล, Streptococcus agalactiae",,,,,,,,,,,,,,,
1,534566,yes,nrms,29419,29419,การพัฒนาและการผลิตวัคซีนทางปากเพื่อการเลี้ยงปล...,project,,,"วัคซีนทางปาก, ปลานิล, Streptococcus agalactiae",,,,,,,,,,,,,,,
2,535120,yes,nrms,2006,2006,การย้ายอาณาเขตของลูกนกและความสามารถในการผลิตลู...,project,,,การรอดของลูกนก การย้ายอาณาเขตของลูกนก การติดวิ...,,,,,,,,,,,,,256109A3060066,,
3,536305,yes,nrms,254239,254239,การศึกษาการทำงานของเบตากลูโคไซเดสจากข้าวที่มีผ...,project,,,"ข้าว,ฮอร์โมนพืช,เบตากลูโคไซเดส,กลูโคส เอสเตอร์...",,,,,,,,,,,,,2560A30402173,,
4,536409,yes,nrms,255170,255170,การพัฒนาวัสดุเคลือบอัลลอยคอมพอสิตโลหะผสมนิกเกิ...,project,,,"เคลือบนิกเกิล-ทังสเตน-โบรอน,การชุบไฟฟ้าแบบร่วม...",,,,,,,,,,,,,2560A10102005,,


## Create Publish Other

In [53]:
result_other = result_other.rename(columns={"id": "auth_ID", "first_author": "firstAuthor","_id":"source_ID","_source.id":"paper_ID","_source.name":"paperTitle","_source.word":"paperKeywords","_source.link":"citation"})

In [54]:
result_other = result_other.drop(['_index', '_type', '_score','first_name',
       '_source.author','_source.start','_source.stop','_source.kind','_source.group',
       '_source.goal','_source.branch','_source.status','_source.budget','_source.department',
       '_source.pullDate','new_name','language'],axis=1)

In [55]:
result_other['sourceName'] = 'nrms'
result_other['paperType'] = 'project'

# result_other['paper_ID']=''
# result_other['paperTitle']=''
result_other['paperAbtract_th'] = ''
result_other['paperAbtract_en'] = ''
# result_other['paperKeywords'] = ''
result_other['paperPage'] = ''
result_other['publishedDate'] = ''
result_other['publishedYear'] = ''
result_other['publisherName'] = ''
result_other['bookName'] = ''
result_other['bookNo'] = ''
result_other['bookVolume'] = ''

result_other['bookEditor'] = ''
result_other['bookSeries'] = ''
result_other['Fund'] = ''
result_other['DOI'] = ''
result_other['ISSN'] = ''
# result_other['citation'] = ''
result_other['refURL']=''
result_other['crossRef'] = ''

In [56]:
all_column =  ["auth_ID","firstAuthor","sourceName","source_ID","paper_ID","paperTitle","paperType","paperAbtract_th","paperAbtract_en","paperKeywords","paperPage","publishedDate","publishedYear","publisherName","bookName","bookNo","bookVolume","bookEditor","bookSeries","Fund","DOI","ISSN","citation","refURL","crossRef"]
len(all_column)

25

In [57]:
len(result_other.columns)

25

In [58]:
result_other = result_other[all_column]

In [59]:
result_other

,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef
0,534308,no,nrms,11775,11775,สร้างและพัฒนาศักยภาพของเจ้าหน้าที่กรมควบคุมโรค...,project,,,วัณโรค วิจัย เครือข่ายวิจัย การพัฒนาศักยภาพ กา...,,,,,,,,,,,,,,,
1,534308,no,nrms,12698,12698,การพัฒนาเครื่องปรับระดับเมนิสคัสแบบอัตโนมัติ,project,,,เครื่องแก้ววัดปริมาตร ปิเปต เมนิสคัส การประมวล...,,,,,,,,,,,,,,,
2,534308,no,nrms,16150,16150,การยกระดับโครงสร้างพื้นฐานความปลอดภัยทางชีวภาพ...,project,,,"ความปลอดภัยทางชีวภาพ, ห้องปฏิบัติการวิจัย, โคร...",,,,,,,,,,,,,,,
3,534308,no,nrms,16384,16384,โครงการพัฒนามาตรการปฏิบัติต่อผู้กระทำผิดอาญาแท...,project,,,มาตรการปฏิบัติต่อผู้กระทำผิดอาญาแทนการควบคุมตัว,,,,,,,,,,,,,,,
4,534308,no,nrms,17505,17505,โครงการยกระดับมาตรฐานเครื่องมือทางการแพทย์,project,,,"ทดสอบ, สอบเทียบ, เครื่องมือแพทย์",,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,534308,no,nrms,5934,5934,การจัดตั้งห้องปฏิบัติการตรวจวิเคราะห์อายุด้วยร...,project,,,,,,,,,,,,,,,,,,
702,534308,no,nrms,271519,271519,การพัฒนาการเพาะเลี้ยงปลากระแห (Barbonymus schw...,project,,,"ปลากระแห,ชีววิทยาปลา,การเพาะเลี้ยงปลา",,,,,,,,,,,,,2560A00602003,,
703,534308,no,nrms,626770,626770,โครงการวิจัยความหลากหลายทางชีวภาพของพืชผักและส...,project,,,,,,,,,,,,,,,,2559NRCT72022,,
704,534308,no,nrms,544939,544939,ยุทธศาสตร์การพัฒนากลุ่มวิจัยเพื่อการพัฒนาเชิงพ...,project,,,ระบบบริหารจัดการงานวิจัยเพื่อการพัฒนาสังคมและช...,,,,,,,,,,,,,,,


## Create Publish ALL

In [60]:
result_all = result_all.rename(columns={"id": "auth_ID", "first_author": "firstAuthor","_id":"source_ID","_source.id":"paper_ID","_source.name":"paperTitle","_source.word":"paperKeywords","_source.link":"citation"})

In [61]:
result_all = result_all.drop(['_index', '_type', '_score','first_name',
       '_source.author','_source.start','_source.stop','_source.kind','_source.group',
       '_source.goal','_source.branch','_source.status','_source.budget','_source.department',
       '_source.pullDate','new_name','language'],axis=1)

In [62]:
result_all['sourceName'] = 'nrms'
result_all['paperType'] = 'project'

# result_all['paper_ID']=''
# result_all['paperTitle']=''
result_all['paperAbtract_th'] = ''
result_all['paperAbtract_en'] = ''
# result_all['paperKeywords'] = ''
result_all['paperPage'] = ''
result_all['publishedDate'] = ''
result_all['publishedYear'] = ''
result_all['publisherName'] = ''
result_all['bookName'] = ''
result_all['bookNo'] = ''
result_all['bookVolume'] = ''

result_all['bookEditor'] = ''
result_all['bookSeries'] = ''
result_all['Fund'] = ''
result_all['DOI'] = ''
result_all['ISSN'] = ''
# result_all['citation'] = ''
result_all['refURL']=''
result_all['crossRef'] = ''

In [63]:
all_column =  ["auth_ID","firstAuthor","sourceName","source_ID","paper_ID","paperTitle","paperType","paperAbtract_th","paperAbtract_en","paperKeywords","paperPage","publishedDate","publishedYear","publisherName","bookName","bookNo","bookVolume","bookEditor","bookSeries","Fund","DOI","ISSN","citation","refURL","crossRef"]
len(all_column)

25

In [64]:
len(result_all.columns)

25

In [65]:
result_all = result_all[all_column]

In [66]:
result_all

,auth_ID,firstAuthor,sourceName,source_ID,paper_ID,paperTitle,paperType,paperAbtract_th,paperAbtract_en,paperKeywords,paperPage,publishedDate,publishedYear,publisherName,bookName,bookNo,bookVolume,bookEditor,bookSeries,Fund,DOI,ISSN,citation,refURL,crossRef
0,534267,yes,nrms,7072,7072,การออกแบบกระบวนการผลิตเพื่อการแปรรูปผลิตภัณฑ์จ...,project,,,หอมแดง เครื่องซอยหัวหอมแดง และเครื่องปอกเปลือก...,,,,,,,,,,,,,,,
1,534268,yes,nrms,7073,7073,ระบบช่วยถ่ายภาพสำหรับผู้พิการทางสายตา,project,,,ถ่ายภาพ คนตาบอด ระบบช่วยเหลือ,,,,,,,,,,,,,,,
2,534269,yes,nrms,6712,6712,การผลิตแท่งเชื้อเพลิงชีวมวลจากทางปาล์มน้ำมัน,project,,,"ทางปาล์มน้ำมัน, แท่งเชื้อเพลิงชีวมวล, พลังงานท...",,,,,,,,,,,,,,,
3,534270,yes,nrms,7087,7087,เครื่องอบแห้งปลาด้วยแสงอินฟาเรดเพื่อผลิตภัณฑ์โ...,project,,,"ปลาอบแห้ง, อินฟาเรด, หนึ่งตำบลหนึ่งผลิตภัณฑ์",,,,,,,,,,,,,,,
4,534271,yes,nrms,9502,9502,การพัฒนาเครื่องคัดขนาดเมล็ดบัวหลวง.,project,,,"บัวหลวง, เมล็ดบัวหลวง, เครื่องคัดขนาด",,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27310,547924,yes,nrms,640818,640818,การสื่อสารการตลาดแบบบูรณาการเพื่อรณรงค์ดูแลผู้...,project,,,การสื่อสารการตลาดแบบบูรณาการ การดูแลผู้สูงอายุ...,,,,,,,,,,,,,2558000013130,,
27311,536246,yes,nrms,303080,303080,การวิจัยและถ่ายทอดเทคโนโลยีชุดถังหมักคุณภาพสูง...,project,,,การวิจัยและถ่ายทอดเทคโนโลยี ชุดถังหมักควบคุมแบ...,,,,,,,,,,,,,2558000111024,,
27312,541420,yes,nrms,303082,303082,การศึกษาและสื่อสารของวิสาหกิจชุมชนข้าวไร่แปลงย...,project,,,กระบวนการสื่อสาร ข้าวไร่ในแปลงยางพารา ข้าวอินท...,,,,,,,,,,,,,2558000112271,,
27313,536886,yes,nrms,383660,383660,ส่วนผสมของน้ำประสานเงินต่อการเชื่อมของเครื่องป...,project,,,,,,,,,,,,,,,,2558000112589,,


## df_author

In [67]:
df_unique_name['language'] = df_unique_name['new_name'].apply(language)

In [68]:
result_author_thai = df_unique_name[df_unique_name['language']=='th']

result_author_eng = df_unique_name[df_unique_name['language']=='eng']

result_author_other = df_unique_name[df_unique_name['language']=='other']

In [69]:
result_author_thai

,new_name,id,language
0,นายศิวกร อ่างทอง,534267,th
1,ผศ. อำนวย เรืองวารี,534268,th
2,นายประสิทธิ์ ศรีนคร,534269,th
3,ผศ.ดร. ชัยยะ ปราณีตพลกรัง,534270,th
4,รศ. ดร. จตุรงค์ ลังกาพินธุ์,534271,th
...,...,...,...
13654,นายจักรกฤษณ์ พจนศิลป์,547921,th
13655,นายเติมเพชร สุขคณาภิบาล,547922,th
13656,นางนัทธีรัตน์ พีระพันธุ์,547923,th
13657,นายอดิศักดิ์ จำปาทอง,547924,th


In [70]:
result_author_eng

,new_name,id,language
298,Assist. Prof. Dr.,534565,eng
299,Dr. Hien Van Doan,534566,eng
853,นายAndrew Pierce,535120,eng
2038,Prof.Dr. James Ketudat-Cairns,536305,eng
2142,Dr. เจียเชียน ฉิน,536409,eng
2746,Dr. กรวรรณ ศรีงาม,537013,eng
3182,อ่าน Table Title_monk เพิ่ม โด เฉลิมชัย ชยเมธีกุล,537449,eng
3637,นายJames C. Moran,537904,eng
3785,นายYupeng Yan,538052,eng
4364,Dr.,538631,eng


In [71]:
result_author_other

,new_name,id,language
41,,534308,other


# Export

In [72]:
result_author_thai = df_unique_name[df_unique_name['language']=='th']

result_author_eng = df_unique_name[df_unique_name['language']=='eng']

result_author_other = df_unique_name[df_unique_name['language']=='other']

result_thai.index.name = 'id'
result_eng.index.name = 'id'
result_other.index.name = 'id'
result_all.index.name = 'id'

result_thai.to_csv('nrms_publish_thai.csv')
result_eng.to_csv('nrms_publish_eng.csv')
result_other.to_csv('nrms_publish_other.csv')
result_all.to_csv('nrms_publish_all.csv')

result_author_thai[['id','new_name']].to_csv('nrms_author_thai.csv')
result_author_eng[['id','new_name']].to_csv('nrms_author_eng.csv')
result_author_other[['id','new_name']].to_csv('nrms_author_other.csv')